In [1]:
from ngstents._pytents import TentSlab
from ngstrefftz import TWave
from ngsolve.TensorProductTools import *
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
import time

importing NGSolve-6.2.2201-15-gf56194365


# Trefftz + tent pitching

## Constructing a tent-pitched mesh

In [2]:
import netgen.geom2d as ngeom2d
def Lmesh(maxh):
    mu = 0.4
    mp = MeshingParameters (maxh = maxh)
    minh = math.pow(maxh,1/mu)
    refpoints = int(1/minh)
    for i in range(0, refpoints+1):
        for j in range(0, refpoints+1):
            xk = (i/refpoints-0.5)*2
            yk = (j/refpoints-0.5)*2
            r = sqrt(xk*xk+yk*yk)
            mp.RestrictH (x=xk, y=yk, z=0, h=max(minh, maxh*math.pow(r,1-mu)) )

    geo = ngeom2d.SplineGeometry()
    p1 = geo.AppendPoint (-1,-1)
    p2 = geo.AppendPoint (0,-1)
    p3 = geo.AppendPoint (0,0)
    p4 = geo.AppendPoint (1,0)
    p5 = geo.AppendPoint (1,1)
    p6 = geo.AppendPoint (-1,1)
    geo.Append (["line", p1, p2])
    geo.Append (["line", p2, p3])
    geo.Append (["line", p3, p4])
    geo.Append (["line", p4, p5])
    geo.Append (["line", p5, p6])
    geo.Append (["line", p6, p1])

    return Mesh(geo.GenerateMesh(mp=mp))

maxh = 0.13
mesh = Lmesh(maxh)
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201-15-gf56194365', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'd…

BaseWebGuiScene

In [3]:
import tentswebgui as twg
dt = 0.05   # tent slab height (in time)
wavespeed = 1
ts = TentSlab(mesh, method="edge")
ts.SetMaxWavespeed(wavespeed)
ts.PitchTents(dt=dt, local_ct=True)
print("max slope", ts.MaxSlope())
print("n tents", ts.GetNTents())
twg.Draw(ts)

max slope 0.9999999999999999
n tents 742
Created slab pitcher
Initialised mesh data
Computed reference heights
Setting ready vertices
Pitching tents...


NGSTentsWebGuiWidget(value={'ntents': 742, 'nlayers': 31, 'ngsolve_version': '6.2.2201-15-gf56194365', 'tent_e…

## Trefftz-DG for the acoustic wave equation + Tents

In [4]:
for i in range(0,len(mesh.GetBoundaries())):
    mesh.ngmesh.SetBCName(i,"neumann")

order=4
delta = 500
ee = exp(-(x+1)**2*delta )
initc=((ee,-2*(x+1)*delta * ee, 0, 0))
bdd = CoefficientFunction(( 0,0,0,0))

TT=TWave(order,ts,CoefficientFunction(wavespeed))
TT.SetInitial(initc)
TT.SetBoundaryCF(bdd)

V = L2(mesh,order=order)
u = GridFunction(V,"u")
scene = Draw(u,autoscale=False,min=0,max=0.3)#,deformation=True
t=0
SetNumThreads(4)
with TaskManager():
    while t < 1.5:
        TT.Propagate()
        t += dt
        TT.GetWave(u)
        scene.Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2201-15-gf56194365', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

## Quasi-Trefftz + Tents 

In [5]:
order = 4
delta=500
dt=0.05
ee = exp(-(x)**2*delta )
initc=((ee,-2*(x)*delta * ee, 0, 0))
bdd = CoefficientFunction(( 0,0,0,0))
mesh = Mesh(unit_square.GenerateMesh(maxh = 0.05))
for i in range(0,len(mesh.GetBoundaries())):
    mesh.ngmesh.SetBCName(i,"neumann")
    
ts = TentSlab(mesh, method="edge")
ts.SetMaxWavespeed(y+1)
ts.PitchTents(dt=dt, local_ct=True)
TT=TWave(order,ts,CoefficientFunction(y+1))
TT.SetInitial(initc)
TT.SetBoundaryCF(bdd)

V = L2(mesh,order=order)
u = GridFunction(V,"u")
scene = Draw(u,autoscale=False,min=0,max=0.3)#,deformation=True
t=0
SetNumThreads(4)
with TaskManager():
    while t < 0.5:
        TT.Propagate()
        t += dt
        TT.GetWave(u)
        scene.Redraw()

Created slab pitcher
Initialised mesh data
Computed reference heights
Setting ready vertices
Pitching tents...


WebGuiWidget(value={'ngsolve_version': '6.2.2201-15-gf56194365', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…